In [ ]:
import pypsa
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import geopandas as gpd
from pypsa.plot import add_legend_lines, add_legend_patches, add_legend_semicircles
import yaml



In [ ]:
fn = 'resources/DK_test/networks/base_s_2__12h_2050.nc'
n = pypsa.Network(fn)

In [ ]:
cf_onshorewind=n.generators_t['p_max_pu'].loc[:, n.generators_t['p_max_pu'].columns.astype(str).str.contains('onwind')].sum()
cf_solar=n.generators_t['p_max_pu'].loc[:, n.generators_t['p_max_pu'].columns.astype(str).str.contains('solar')].sum().sort_values(ascending=False)



In [ ]:
n.add("Bus",
      name="DK0 cluster",
      v_nom=n.buses.at["DK0 0", "v_nom"],
      x=n.buses.at["DK0 0", "x"],
      y=n.buses.at["DK0 0", "y"],
      unit=n.buses.at["DK0 0", "unit"],
      location=n.buses.at["DK0 0", "location"],
      country="DK",
      carrier='AC',
      control=n.buses.at["DK0 0", "control"],
      substation_lv=n.buses.at["DK0 0", "substation_lv"],
      substation_off=n.buses.at["DK0 0", "substation_off"],
      )

n.add("Bus",
      name="DK0 cluster H2",
      country="DK",
      x=n.buses.at["DK0 0", "x"],
      y=n.buses.at["DK0 0", "y"],
      location=n.buses.at["DK0 0", "location"],
      unit=n.buses.at["DK0 0 H2", "unit"],
      carrier='H2')



In [ ]:
n.add(
        "Link",
        name="DK0 cluster H2 Electrolysis",
        bus1="DK0 cluster H2",
        bus0="DK0 cluster",
        p_nom_extendable=True,
        carrier="H2 Electrolysis",
        efficiency=n.links.at["DK0 0 H2 Electrolysis", "efficiency"],
        capital_cost=float(n.links.at["DK0 0 H2 Electrolysis", "capital_cost"]),
        marginal_cost=float(n.links.at["DK0 0 H2 Electrolysis", "marginal_cost"]),
        lifetime=n.links.at["DK0 0 H2 Electrolysis", "lifetime"],
        reversed=False,
        overwrite=True,
    )

n.add(
        "Link",
        name="DK0 cluster methanolization",
        bus0="DK0 cluster H2",
        bus1="EU methanol",
        bus2="DK0 cluster",
        bus3="DK0 0 co2 stored",
        bus4=n.links.at["DK0 0 methanolisation", "bus4"],
        carrier="methanolisation",
        p_nom_extendable=True,
        p_min_pu=n.links.at["DK0 0 methanolisation", "p_min_pu"],
        capital_cost=n.links.at["DK0 0 methanolisation", "capital_cost"],
        marginal_cost=n.links.at["DK0 0 methanolisation", "marginal_cost"],
        lifetime=n.links.at["DK0 0 methanolisation", "lifetime"],
        efficiency=n.links.at["DK0 0 methanolisation", "efficiency"],
        efficiency2=n.links.at["DK0 0 methanolisation", "efficiency2"],
        efficiency3=n.links.at["DK0 0 methanolisation", "efficiency3"],
        efficiency4=n.links.at["DK0 0 methanolisation", "efficiency4"],
        reversed=False,
        overwrite=True,
    )

# n.remove(
#         "Link",
#         name="DK0 cluster electricity",
#     )

# n.add(
#         "Link",
#         name="DK0 cluster electricity",
#         bus0="DK0 cluster",
#         bus1="DK0 0",
#         carrier="AC",   
#         p_nom_extendable=True,
#         efficiency=1.0,
#         capital_cost=0.0,
#         marginal_cost=0.0,
#     )

n.add(
        "Generator",
        name="DK0 0 cluster 4 onwind",
        bus="DK0 cluster",
        carrier="onwind",
        p_nom_max=n.generators.at["DK0 0 4 onwind", "p_nom_max"],
        p_max_pu=n.generators.at["DK0 0 4 onwind", "p_max_pu"],
        marginal_cost=n.generators.at["DK0 0 4 onwind", "marginal_cost"],
        capital_cost=n.generators.at["DK0 0 4 onwind", "capital_cost"],
        efficiency=n.generators.at["DK0 0 4 onwind", "efficiency"],
        p_nom_extendable=True,)

In [ ]:
n.generators_t['p_max_pu']['DK0 0 cluster 4 onwind'] = n.generators_t['p_max_pu']['DK0 0 4 onwind']


In [ ]:
n.links["reversed"] = n.links["reversed"].fillna(False).astype(bool)


In [ ]:
n.links.loc[n.links["carrier"]=='methanolisation']

In [ ]:
n.buses

In [ ]:
n.generators
n.generators_t['p_max_pu']


In [ ]:
n.links

In [ ]:
n.export_to_netcdf(fn)
